In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys, time, pickle, math
import scipy

%matplotlib inline

In [ ]:
def import_result_penalizedEM(fname):
    with open(fname, "rb") as p:
        res1 = pickle.load(p)
    pi = [r["pi"] for r in res1]
    mu = [r["mu"] for r in res1]
    cov = [r["cov"] for r in res1]
    p_loss = [r["p_loss"] for r in res1]
    d_loss = [r["d_loss"] for r in res1]
    iters = [r["iters"] for r in res1]
    time = [r["time"] for r in res1]
    return pi, mu, cov, p_loss, d_loss, iters, time

def import_result_EM(fname):
    with open(fname, "rb") as p:
        res1 = pickle.load(p)
    pi = [r["pi"] for r in res1]
    mu = [r["mu"] for r in res1]
    conv = [r["conv"] for r in res1]
    loss = [r["loss"] for r in res1]
    iters = [r["iters"] for r in res1]
    time = [r["time"] for r in res1]
    init_guess = [r["init_guess"] for r in res1]
    return pi, mu, conv, loss, iters, time, init_guess

def import_result_GD(fname):
    res = np.load(fname)
    pi = res["pi"]
    mu = res["mu"]
    cov = res["cov"]
    p_loss = res["train_p_losses"]
    d_loss = res["train_d_losses"]
    return pi, mu, cov, p_loss, d_loss

In [ ]:
def MoG_prob(x, pi, mu, cov=None):
    K, dim = mu.shape
    if cov is None:
        cov = np.tile(np.eye(dim), (K, 1, 1))
    assert x.shape == (dim,)
    assert pi.shape == (K,)
    assert cov.shape == (K, dim, dim)
    prob = 0.0
    for k in range(K):
        s, logdet = np.linalg.slogdet(cov[k] + np.eye(dim) * 1e-3)
        # print(s, np.exp(logdet))
        assert s > 0
        log_prob_k = -dim * 0.5 * math.log(2 * math.pi) - 0.5 * logdet - 0.5 * (x - mu[k]).dot(x - mu[k])
        prob += np.exp(log_prob_k) * pi[k]
    return prob

def MoG_plot(pi, mu, cov=None):
    plt.figure(figsize=(5, 5))
    
    x1 = np.linspace(-2.0, 8.0, 101)
    x2 = np.linspace(-2.0, 8.0, 101)
    p_lists = []
    for _x2 in x2:
        p = []
        for _x1 in x1:
            p.append(MoG_prob(np.array((_x1, _x2)), pi, mu, cov) )
        p_lists.append(p)
    P = np.array(p_lists)
    
    plt.imshow(P, origin='lower', interpolation='bilinear')
    plt.xticks(np.linspace(0, 100, 6), np.linspace(-2, 8, 6))
    plt.yticks(np.linspace(0, 100, 6), np.linspace(-2, 8, 6))

def calc_KL(P, Q, x1_linspace, x2_linspace=None):
    # KL(P, Q)
    if x2_linspace is None:
        x2_linspace = x1_linspace
        
    points = [np.array((x1, x2)) for x2 in x2_linspace for x1 in x1_linspace]
    P_probs = [P(x) for x in points]
    Q_probs = [Q(x) for x in points]
    KL = scipy.stats.entropy(P_probs, Q_probs)
    return KL

## Temp experiments

In [ ]:
pi, mu, cov, p_loss, d_loss, iters, time = import_result_penalizedEM("results/multi-adv-0/EM/Penalized-K=10-lam=10.0-N=1000.p")

In [ ]:
mu[0]

In [ ]:
pi[0]

In [ ]:
cov[0]

In [ ]:
print(MoG_prob(np.array((1, 3)), pi[0], mu[0], cov[0]))
print(MoG_prob(np.array((3, 1)), pi[0], mu[0], cov[0]))
print(MoG_prob(np.array((3, 5)), pi[0], mu[0], cov[0]))
print(MoG_prob(np.array((5, 3)), pi[0], mu[0], cov[0]))
# print(MoG_prob(np.array((5, 5)), pi[0], mu[0], cov[0]))

In [ ]:
plt.style.use('default')
MoG_plot(pi[0], mu[0], cov[0])

In [ ]:
time

## Losses

In [ ]:
pi, mu, cov, p_loss, d_loss, iters, time = import_result_penalizedEM("results/multi-adv-0/EM/Penalized-K=10-lam=10.0-N=1000.p")

In [ ]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure()
ax = plt.axes()

plt.title("Loss")
plt.xlabel("EM Iteration")
plt.ylabel("loss");
x = np.linspace(0, 199, 200)
for loss in d_loss:
    ax.plot(x[:len(loss)-1], loss[1:])

In [ ]:
fig = plt.figure()
ax = plt.axes()

plt.ylim(-1, 15);
plt.title("Inner Loop iterations")
plt.xlabel("EM Iteration")
plt.ylabel("# of normalization iteration");
x = np.linspace(0, 199, 200)
for i in iters:
    ax.plot(x[:len(i)], i)

## True Distribution

In [ ]:
data_fname = 'datasets/multi-adv-0/data_multi_adv.npz'
load_data = np.load(data_fname)
# true_pi = load_data['pi']
true_pi = np.ones(5) / 5.0
true_mu = load_data['mu']
samples = load_data['samples']
adv_sample = load_data['adv_sample']

In [ ]:
true_pi

In [ ]:
true_mu

In [ ]:
print(MoG_prob(np.array((1, 3)), true_pi, true_mu))
print(MoG_prob(np.array((3, 1)), true_pi, true_mu))
print(MoG_prob(np.array((3, 5)), true_pi, true_mu))
print(MoG_prob(np.array((5, 3)), true_pi, true_mu))
# print(MoG_prob(np.array((5, 5)), true_pi, true_mu))

In [ ]:
plt.style.use('default')
MoG_plot(true_pi, true_mu)

## Pen-EM Results

In [ ]:
#K=5-lam=0.1
pi, mu, cov, p_loss, d_loss, iters, time = import_result_penalizedEM("results/multi-adv-0/EM/Penalized-K=10-lam=10.0-N=1000.p")
pi

In [ ]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure()
ax = plt.axes()

plt.title("Loss")
plt.xlabel("EM Iteration")
plt.ylabel("loss");
x = np.linspace(0, 200, 200)
for loss in d_loss:
    ax.plot(x[:len(loss)-1], loss[1:])

In [ ]:
fig = plt.figure()
ax = plt.axes()

plt.ylim(-1, 15);
plt.title("Inner Loop iterations")
plt.xlabel("EM Iteration")
plt.ylabel("# of normalization iteration");
x = np.linspace(0, 199, 200)
for i in iters:
    ax.plot(x[:len(i)], i)

In [ ]:
plt.style.use('default')
for i in range(len(pi)):
    MoG_plot(pi[i], mu[i], cov[i])

In [ ]:
_i = 1
MoG_plot(pi[_i], mu[_i], cov[_i])
# plt.savefig('Penalized-K=10-lam=1.0-N=100.png')

In [ ]:
_i = 0
print(MoG_prob(np.array((1, 3)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((3, 1)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((3, 5)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((5, 3)), pi[_i], mu[_i], cov[_i]))

## Original EM Results

In [ ]:
pi, mu, cov, loss, iters, time, init_guess = import_result_EM("results/multi-adv-0/EM/EM-K=10-lam=10.0-N=1000.p")
pi

In [ ]:
plt.style.use('default')
for i in range(len(pi)):
    MoG_plot(pi[i], mu[i], cov[i])

In [ ]:
_i = 0
MoG_plot(pi[_i], mu[_i], cov[_i])
# plt.savefig('EM-K=5-lam=1.0-N=100.png')

In [ ]:
_i = 0
print(MoG_prob(np.array((1, 3)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((3, 1)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((3, 5)), pi[_i], mu[_i], cov[_i]))
print(MoG_prob(np.array((5, 3)), pi[_i], mu[_i], cov[_i]))

## GD Results

In [ ]:
pi, mu, cov, p_loss, d_loss = import_result_GD("results/multi-adv-0/GD/result-adv-adam-K=10-lam=0.1-id=1.npz")
# plt.style.use('default')
# MoG_plot(pi, mu, cov)

In [ ]:
pi

In [ ]:
mu

In [ ]:
cov

In [ ]:
# print(MoG_prob(np.array((1, 1)), pi, mu, cov))
print(MoG_prob(np.array((1, 3)), pi, mu, cov))
print(MoG_prob(np.array((3, 1)), pi, mu, cov))
print(MoG_prob(np.array((3, 5)), pi, mu, cov))
print(MoG_prob(np.array((5, 3)), pi, mu, cov))

In [ ]:
MoG_plot(pi, mu, cov)
# plt.savefig('GD-K=10-lam=1.0-N=100.png')

## KL calculation

In [ ]:
pi_GD, mu_GD, cov_GD, _, _ = import_result_GD("results/multi-adv-0/GD/result-adv-adam-K=10-lam=0.1-id=1.npz")
pi_EM, mu_EM, cov_EM, _, _, _, _ = import_result_penalizedEM("results/multi-adv-0/EM/Penalized-K=10-lam=0.1-N=100.p")

True_P = lambda x : MoG_prob(x, true_pi, true_mu)
GD_P = lambda x : MoG_prob(x, pi_GD, mu_GD, cov_GD)
EM_P = lambda x : MoG_prob(x, pi_EM[0], mu_EM[0], cov_EM[0])

print(True_P(np.array((3, 3))))
print(GD_P(np.array((3, 3))))
print(EM_P(np.array((3, 3))))

In [ ]:
print(calc_KL(GD_P, True_P, np.linspace(-2, 8, 101)))
print(calc_KL(EM_P, True_P, np.linspace(-2, 8, 101)))

In [ ]:
# correct KL: P = true, Q = estimation
print(calc_KL(True_P, GD_P, np.linspace(-2, 8, 101)))
print(calc_KL(True_P, EM_P, np.linspace(-2, 8, 101)))

In [ ]:
plt.style.use('default')
MoG_plot(pi_GD, mu_GD, cov_GD)
MoG_plot(pi_EM[0], mu_EM[0], cov_EM[0])

In [ ]:
K = 3
lam = 0.1
GD_KL_list = []
for i in range(3):
    try:
        pi_GD, mu_GD, cov_GD, _, _ = import_result_GD("results_multi_adv/result-adv-gd-K={}-lam={}-id={}.npz".format(K, lam, i+1))
    except:
        break
    True_P = lambda x : MoG_prob(x, true_pi, true_mu)
    GD_P = lambda x : MoG_prob(x, pi_GD, mu_GD, cov_GD)
    KL = calc_KL(True_P, GD_P, np.linspace(-2, 8, 101))
    print(KL)
    GD_KL_list.append(KL)

GD_KL_avg = sum(GD_KL_list) / len(GD_KL_list)
GD_KL_avg

In [ ]:
lam_settings = [1.0, 10.0]
K_settings = [3, 5, 10]
all_settings = [(K, lam) for lam in lam_settings for K in K_settings]

for K, lam in all_settings:
    print('K = {}, lam = {}'.format(K, lam))
    GD_KL_list = []
    for i in range(3):
        try:
            pi_GD, mu_GD, cov_GD, _, _ = import_result_GD("results/multi-adv-0/GD/result-adv-gd-K={}-lam={}-id={}.npz".format(K, lam, i+1))
        except:
            break
        True_P = lambda x : MoG_prob(x, true_pi, true_mu)
        GD_P = lambda x : MoG_prob(x, pi_GD, mu_GD, cov_GD)
        KL = calc_KL(True_P, GD_P, np.linspace(-2, 8, 101))
        print(KL)
        GD_KL_list.append(KL)
    if len(GD_KL_list) > 0:
        GD_KL_avg = sum(GD_KL_list) / len(GD_KL_list)
        print('GD_KL_avg = {}'.format(GD_KL_avg))

    EM_KL_list = []
    pi_EM, mu_EM, cov_EM, _, _, _, _ = import_result_penalizedEM("results/multi-adv-0/EM/Penalized-K={}-lam={}-N=100.p".format(K, lam))
    for i in range(len(pi_EM)):
        True_P = lambda x : MoG_prob(x, true_pi, true_mu)
        EM_P = lambda x : MoG_prob(x, pi_EM[i], mu_EM[i], cov_EM[i])
        KL = calc_KL(True_P, EM_P, np.linspace(-2, 8, 101))
        print(KL)
        EM_KL_list.append(KL)

    EM_KL_avg = sum(EM_KL_list) / len(EM_KL_list)
    print('PenEM_KL_avg = {}'.format(EM_KL_avg))
    
    EM_KL_list = []
    pi_EM, mu_EM, cov_EM, _, _, _, _ = import_result_EM("results/multi-adv-0/EM/EM-K={}-lam={}-N=100.p".format(K, lam))
    for i in range(len(pi_EM)):
        True_P = lambda x : MoG_prob(x, true_pi, true_mu)
        EM_P = lambda x : MoG_prob(x, pi_EM[i], mu_EM[i], cov_EM[i])
        KL = calc_KL(True_P, EM_P, np.linspace(-2, 8, 101))
        print(KL)
        EM_KL_list.append(KL)

    EM_KL_avg = sum(EM_KL_list) / len(EM_KL_list)
    print('EM_KL_avg = {}'.format(EM_KL_avg))